In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from numpy import zeros_like
from IPython.display import display
def display_matrix(m):
    display(sympy.Matrix(m))
import sympy
import pandas as pd
import scipy.stats as ss

sympy.init_printing()

In [2]:
def gbm_paths_sample(S0, mu, sigma, T, I, paths):
    np.random.seed(42)
    S0, paths, I, T = S0, paths, I, T
    dt = T / I
    matrix = np.zeros((paths, I))
    save_normal = np.zeros((paths, I))
    for k in range(paths):
        S = np.zeros(I)
        S[0] = S0
        for i in range(1, I):
            Z = np.random.standard_normal()
            S[i] = S[i-1] * np.exp((mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
        matrix[k] = S
        save_normal[k] = Z
    mean_normal = np.mean(save_normal)
    std_normal = np.std(save_normal)
    return mean_normal, std_normal

In [3]:
basis = 'leguerre'

# parameters

mu = 0.06
r = 0.06
T = 1
I = 250
paths = 10000
S0 = 40
S0_list = range(37, 43, 1)
m = 0
K = 40

GBM_sigma = 0.04
GBM_mu = r

LN_lam = 1
LN_sigma = 0.02
LN_mu = r
LN_v = 0.02

JR_lam = 0.01
JR_sigma = 0.03
JR_mu = r + JR_lam

LNparams = (LN_lam, LN_sigma, LN_mu, LN_v, m)
JRparams = (JR_lam, JR_sigma, JR_mu, 0, m)
GBMparams = (GBM_mu, GBM_sigma)

In [4]:
mean_normal, std_normal = gbm_paths_sample(S0, GBM_mu, GBM_sigma, T, I, paths)

In [5]:
def gbm_paths_mm(S0, mu, sigma, T, I, paths, mean_normal, std_normal):
    np.random.seed(42)
    S0, paths, I, T = S0, paths, I, T
    dt = T / I
    matrix = np.zeros((paths, I))
    for k in range(paths):
        S = np.zeros(I)
        S[0] = S0
        for i in range(1, I):
            Z = np.random.standard_normal()
            S[i] = S[i-1] * np.exp((mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * ((Z-mean_normal)/std_normal))
        matrix[k] = S
    return matrix

In [7]:
def LSM(K, S, I, df, basis, deg):
    paths = len(S)
    np.random.seed(42)
    H = np.maximum(K - S, 0)  # intrinsic values for put option
    V = np.zeros_like(H)  # value matrix
    V[:, -1] = H[:, -1]  # set value at maturity equal to intrinsic value

    # Valuation by LS Method
    for t in range(I - 2, 0, -1):
        good_paths = H[:, t] > 0  # paths where the intrinsic value is positive

        if np.sum(good_paths) > 0:
            if basis == 'poly':
                rg = np.polyfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polyval(rg, S[good_paths, t])
            elif basis == 'legendre':
                rg = np.polynomial.legendre.legfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.legendre.legval(S[good_paths, t], rg)
            elif basis =='laguerre':
                rg = np.polynomial.laguerre.lagfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.laguerre.lagval(S[good_paths, t], rg)
            else:  # 'hermite'
                rg = np.polynomial.hermite.hermfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.hermite.hermval(S[good_paths, t], rg)

            exercise = np.zeros(len(good_paths), dtype=bool)
            exercise[good_paths] = H[good_paths, t] > C
        else:
            # If all intrinsic values are zero, mark all as non-exercise
            exercise = np.zeros(len(good_paths), dtype=bool)

        V[exercise, t] = H[exercise, t]
        V[exercise, t + 1 :] = 0
        discount_path = ~exercise
        V[discount_path, t] = V[discount_path, t + 1] * df

    V0 = np.mean(V[:, 1]) * df  # discounted expectation of V[t=1]
    V0_array = V[:, 1] * df
    SE = np.std(V[:, 1] * df) / np.sqrt(paths)
    variance = np.var(V[:, 1] * df)
    return V0, V0_array, SE, variance

In [8]:
def gbm_paths(S0, mu, sigma, T, I, paths):
    #set seed
    np.random.seed(42)
    S0, paths, I, T = S0, paths, I, T
    dt = T / I
    matrix = np.zeros((paths, I))
    for k in range(paths):
        S = np.zeros(I)
        S[0] = S0
        for i in range(1, I):
            Z = np.random.standard_normal()
            S[i] = S[i-1] * np.exp((mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
        matrix[k] = S
    return matrix

In [9]:
paths_mm = gbm_paths_mm(S0, GBM_mu, GBM_sigma, T, I, paths, mean_normal, std_normal)
paths = gbm_paths(S0, GBM_mu, GBM_sigma, T, I, paths)

V0_mm, V0_array_mm, SE_mm, variance_mm = LSM(K, paths_mm, I, np.exp(-r * T), basis, 3)
V0, V0_array, SE, variance = LSM(K, paths, I, np.exp(-r * T), basis, 3)

print(SE_mm.round(5), variance_mm.round(5))
print(SE.round(5), variance.round(5))

0.00083 0.00694
0.00085 0.0072
